In [15]:
from collections import OrderedDict
from itertools import groupby, chain, islice
import re
import csv

### islice(g, 2) e.g. booosst -> boosst, juuuuuust -> juust
### TODO: after filter once, still need some operations to make juust, damnn, pleaasee -> just, damn, please
def replaceRepeatedChar(sentence):
    return ''.join(chain.from_iterable(islice(g, 2) for s, g in groupby(sentence.lower())))

def replaceURL(sentence):
    return re.sub(r"http\S+", "|U|", sentence)

### find all tag name (won't be used, just sub function for replaceNameTag)
def findTagName(sentence):
    return re.compile(r'\B@\w+').findall(sentence) 

def replaceNameTag(sentence):
    return re.sub(r'\B@\w+\S+', "|T|", sentence)

def replaceEmotion(sentence):    
    dict = {}
    dict['&lt;3'] = 'Heart'
    dict[':-)'] = 'Positive'
    dict['=)'] = 'Positive'
    dict[':)'] = 'Positive'
    dict[': )'] = 'Positive'
    dict[':o)'] = 'Positive'
    dict[':]'] = 'Positive'
    dict[':3'] = 'Positive'
    dict[':c)'] = 'Positive'
    
    dict[':D'] = 'Extremely Positive'
    dict[':-D'] = 'Extremely Positive'
    dict['C:'] = 'Extremely Positive' 
    dict['=D'] = 'Extremely Positive' 
    dict['XD'] = 'Extremely Positive' 
    dict['LOL'] = 'Extremely Positive' 
    
    dict[':-('] = 'Negative'
    dict[':/'] = 'Negative'
    dict[':('] = 'Negative'
    dict[': ('] = 'Negative'
    dict[':c'] = 'Negative'
    dict[':['] = 'Negative'
    dict['T_T'] = 'Negative'
    dict[':S'] = 'Negative'
    dict[":'S"] = 'Negative'

    dict['D8'] = 'Extremely Negative'
    dict['D;'] = 'Extremely Negative'
    dict['D='] = 'Extremely Negative'
    dict['DX'] = 'Extremely Negative'     
    dict['v.v'] = 'Extremely Negative'
    
    dict[': |'] = 'Neutral'
    
    words = sentence.split()
    for word in words:
        for key in dict.iterkeys():
            if key in word:
                print key
                return 1
    return 0

def replaceEmotion2(sentence):
    words = sentence.split()
    print sentence
    for word in words:
        for key in emojis.iterkeys():
            if key in word.decode("utf8"):
                print key
                print emojis.get(key)
                sentence = sentence.replace(unicode(word, "utf8"),emojis.get(key))
                print sentence
    return sentence

def replaceNegations(sentence):
    negWords = set(open('negative-words.txt').read().split())
    for word in negWords:
        sentence = sentence.replace(word,'Not')
    return sentence

def replacePositive(sentence):
    posWords = set(open('positive-words.txt').read().split())
    #print posWords
    for word in posWords:
        sentence = sentence.replace(word,'Good')
    return sentence

def removeStopwords(sentence):
    for word in stopwords[0]:
        regex = re.compile(r'\b('+word+r')\b')
        sentence = regex.sub("-", sentence.lower())  
    return sentence.strip()

In [16]:
# Construct an set of stopwords
stopwords = []
with open("stop-word-list.csv",'rb') as f:
    fcsv = csv.reader(f, skipinitialspace=True, delimiter=',')
    for row in fcsv:
        stopwords.append(row)

In [8]:
### Test function ###
s1 = "I likeeeee move ittt movee it"
print 'Test replaceRepeatedChar function:'
print s1
print replaceRepeatedChar(s1), '\n'

# second time filter (not sure how to do here, maybe retrieve some degree of frequency and replace them)
print 'Filter those meaningless words:'
## assume after added into dict, some example may fail such as pleaasee -> pleas
meaningless_test = ['likee', 'itt', 'movee', 'juust', 'fuuck', 'pleaasee', 'biigg', 'aahh']
print meaningless_test
print map(lambda x: "".join(OrderedDict.fromkeys(x)), meaningless_test), '\n'

# Replace URL
s2 = 'aaahhh... The taste of Italy http://www.caffevita.com/shop/images/BialettiOldSchool.jpg'
print 'Test replaceURL function:'
print s2
print replaceURL(s2), '\n'

# Replace tag Name
tagName = "  -@otto where are you @@@chelin you are so smart! &it @chienhan let's paly pokemongo"
print 'Test findTagName function(not important):'
print tagName
print findTagName(tagName), '\n'
print 'Test replaceNameTag function:'
print replaceNameTag(tagName), '\n'

# Replace emotion
emojiText = 'I got 100 Pikachus with hats and it is so amazing :3'
print 'Test replaceEmotion function:'
print emojiText
print replaceEmotion(emojiText), '\n'

# Replace negative words
negativeText = 'I type in fumble snags coward horrendous these words'
print 'Test replaceNegations function:'
print negativeText
print replaceNegations(negativeText),'\n'

# Replace positive words
positiveText = 'I can spell accurately beauty glitz!'
print 'Test replacePositive function:'
print positiveText
print replacePositive(positiveText),'\n'

# Remove stopwords
stopword_test = 'no no no I my me we our us Hello Pika!!!'
print 'Test removeStopwords function:'
print stopword_test
print removeStopwords(stopword_test)
### End of Testing ###

Test replaceRepeatedChar function:
I likeeeee move ittt movee it
i likee move itt movee it 

Filter those meaningless words:
['likee', 'itt', 'movee', 'juust', 'fuuck', 'pleaasee', 'biigg', 'aahh']
['like', 'it', 'move', 'just', 'fuck', 'pleas', 'big', 'ah'] 

Test replaceURL function:
aaahhh... The taste of Italy http://www.caffevita.com/shop/images/BialettiOldSchool.jpg
aaahhh... The taste of Italy |U| 

Test findTagName function(not important):
  -@otto where are you @@@chelin you are so smart! &it @chienhan let's paly pokemongo
['@otto', '@chelin', '@chienhan'] 

Test replaceNameTag function:
  -|T| where are you @@|T| you are so smart! &it |T| let's paly pokemongo 

Test replaceEmotion function:
I got 100 Pikachus with hats and it is so amazing :3
:3
1 

Test replaceNegations function:
I type in fumble snags coward horrendous these words
I type in Not Not Not Not these words 

Test replacePositive function:
I can spell accurately beauty glitz!
I can spell Good Good Good! 

Test re

In [9]:
# Construct an emoji dictionary 
import codecs
emojis = dict()
f = codecs.open('emoji_table.txt', 'r', 'UTF-8')
for line in f:
    types = (line.split(","))
    emojis[types[0]] = types[1]
    print types[0].encode("utf8")
    print emojis[types[0]]

emoji
meaning

😄
A_smiley1

😃
A_smiley2

😀
A_smiley3

😊
A_smiley4

☺️
A_blush

😉
A_wink

😍
A_loveeyes

😘
A_kiss1 

😚
A_kiss2

😗
A_kiss3

😙
A_kiss4

😜
A_tongue1

😝
A_tongue2

😛
A_tongue3

😳
A_embarassed

😁
A_cheese

😔
A_sad1

😌
A_sad2

😒
A_sad3

😞
A_sad4

😣
A_sad5

😢
A_sad6

😂
A_sad7

😭
A_sad8

😪
A_sad9

😥
A_sad10

😰
A_sad11

😅
A_sad12

😓
A_sad13

😩
A_sad14

😫
A_sad15

😨
A_worry1

😱
A_worry2

😠
A_angry1

😡
A_angry2

😤
A_angry3

😖
A_wince

😆
A_laugh

😋
A_licklips

😷
A_respirator

😎
A_sunglasses

😴
A_Zzz

😵
A_dead1

😲
A_dead2

😟
A_worry1

😦
A_worry2

😧
A_worry3

😈
A_devil1

👿
A_devil2

😮
A_oh

😬
A_cheese2

😐
A_straight

😕
A_worry4

😯
A_oh2

😶
A_nomouth

😇
A_angel

😏
A_smirk

😑
A_straight2

👲
B_gua

👲🏻
B_gua1

👲🏼
B_gua2

👲🏽
B_gua3

👲🏾
B_gua4

👲🏿
B_gua5

👳
B_turbin

👳🏻
B_turbin1

👳🏼
B_turbin2

👳🏽
B_turbin3

👳🏾
B_turbin4

👳🏿
B_turbin5

👮
B_police

👮🏻
B_police1

👮🏼
B_police2

👮🏽
B_police3

👮🏾
B_police4

👮🏿
B_police5

👷
B_safety

👷🏻
B_safety1

👷🏼
B_safety2

👷🏽
B_safety3

👷🏾
B_safety4

👷🏿
B_saf

In [14]:
import csv
import string

punctuation = set(string.punctuation)
data_text = []
data_sentiment = []
count = 0
emoji = 0
data_emoji = []
### modify the data processing here

with open('Sentiment Analysis Dataset.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        ## Skip header
        if row[1] == "Sentiment":
            continue
        data_sentiment.append(float(row[1]))
        if count <= 100000:
            if replaceEmotion(row[3]) >= 1:
                emoji += 1
        else:
            break;
        count += 1
        ### Ignore capitalization and remove punctuation
        r = ''.join([c for c in row[3].lower() if not c in punctuation])
        data_text.append(r.split())

print "Data size:", len(data_sentiment)
print "Number of Emoji", emoji

:3
T_T
&lt;3
&lt;3
:]
&lt;3
&lt;3
&lt;3
=D
:[
:-D
&lt;3
&lt;3
&lt;3
=D
&lt;3
&lt;3
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
=D
=D
&lt;3
:3
&lt;3
:3
:[
&lt;3
&lt;3
:3
:]
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:[
:]
&lt;3
&lt;3
&lt;3
:-D
&lt;3
&lt;3
&lt;3
=D
C:
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:]
&lt;3
:3
&lt;3
&lt;3
&lt;3
DX
:-D
&lt;3
:3
&lt;3
&lt;3
&lt;3
=D
&lt;3
:[
&lt;3
&lt;3
&lt;3
DX
&lt;3
&lt;3
&lt;3
&lt;3
T_T
:3
D;
:3
:]
:c
:3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
D;
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
=D
&lt;3
&lt;3
:3
&lt;3
:]
&lt;3
=D
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
&lt;3
C:
&lt;3
&lt;3
&lt;3
:[
&lt;3
&lt;3
&lt;3
:o)
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:]
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
DX
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
=D
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
D;
:[
&lt;3
&lt;3
:3
&lt;

In [ ]:
# check first data point
print data_text[0]
print data_sentiment[0]

In [ ]:
import numpy as np
import scipy.optimize
from collections import defaultdict
from sklearn import linear_model

### Producing feature using word count

wordCount = defaultdict(int)

for x in data_text:
    for y in x:
        wordCount[y] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
print "Number of unique words" , len(counts)
print ""
print "Popular words", counts[:10]        

words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [ ]:
### modify here to produce desire feature

def feature(datum):
    feat = [0]*len(words)
    for w in datum:
        if w in wordSet:
            feat[wordId[w]] += 1.0
    feat.append(1.0) #offset
    return feat

### Sentiment analysis

X = [feature(d) for d in data_text]
y = data_sentiment
K = 50000 ### numbers of training data

#No regularization
#theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

#With regularization
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X[:K], y[:K])
theta = clf.coef_

P = 100000### numbers of prediction data
### here using all data to test accuracy
predictions = clf.predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "\nAccuracy", accuracy

In [7]:

### Logistic regression example 
logisticclf = linear_model.LogisticRegression()
logisticclf.fit(X[:K], y[:K])
predictions = logisticclf.predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "Accuracy", accuracy

Accuracy 0.75398


In [8]:

### Naive Bayes example
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
predictions = gnb.fit(X[:K], y[:K]).predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "Accuracy", accuracy

Accuracy 0.69288


In [ ]:

### SVM example warning: take long time to train, Don't try
'''
from sklearn import svm
svmclf = svm.SVC()
svmclf.fit(X[:K], y[:K]) 
predictions = svmclf.predict.predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "Accuracy", accuracy